In [1]:
import pandas as pd
import numpy as np
import datetime
import glob
import gc
import json
import time

In [2]:
start="2009-10-14"


In [3]:
# aa=datetime.datetime.now()
# bb=aa + datetime.timedelta(days=-3650)
# start=bb

In [4]:
libor=pd.read_csv("../LIBOR USD.csv")

In [5]:
libor["Date"]=libor["Date"].map(lambda x:pd.to_datetime(x,format='%d.%m.%Y'))

In [6]:
libor["Month"]=libor["Date"].map(lambda x:x.to_period("M"))

In [7]:
libor=libor.sort_values("Date")

In [8]:
libor=libor[libor["Date"]>start]
libor=libor.groupby("Month")["ON","1W","1M","2M","3M","6M","12M"].mean()

In [9]:
path_trea =r'../Treasury' 
allFiles_trea = glob.glob(path_trea + "/*.csv")
frame_trea = pd.DataFrame()
list_trea = []
for file_trea in allFiles_trea:
    df_trea = pd.read_csv(file_trea,index_col="Date", header=0)
    list_trea.append(df_trea)
frame_trea=pd.concat(list_trea)

In [10]:
frame_trea.index=frame_trea.index.map(lambda x:pd.to_datetime(x,format='%m/%d/%Y'))

In [11]:
frame_trea["Month"]=frame_trea.index.map(lambda x:x.to_period("M"))

In [12]:
frame_trea.to_csv("Treasury_month.csv")

In [13]:
frame_trea=frame_trea[frame_trea.index>start]
frame_trea=frame_trea.groupby("Month")["1 Mo","6 Mo","1 Yr","5 Yr","10 Yr","20 Yr","30 Yr"].mean()

In [14]:
hud_cat=pd.read_csv("../cat_data.csv")
hud_cat["CURRENT PRINCIPAL AND INTEREST"]=hud_cat["CURRENT PRINCIPAL AND INTEREST"].map(lambda x:float(x.replace(",","")))
hud_cat["DATE"]=hud_cat["INITIAL ENDORSEMENT DATE"].map(lambda x:pd.to_datetime(x,format='%d/%m/%Y'))
hud_cat["Month"]=hud_cat["DATE"].map(lambda x:x.to_period("M"))
hud_cat["coor"]=list(zip(hud_cat.longitude, hud_cat.latitude))

hud=hud_cat.groupby(["PROPERTY STATE","PROPERTY CITY","Month"]).agg({'INTEREST RATE':['mean'],'coor':['first'],'CURRENT PRINCIPAL AND INTEREST':['mean']})
hud=hud.reset_index(["PROPERTY STATE","PROPERTY CITY","Month"])

In [15]:
hud

,PROPERTY STATE,PROPERTY CITY,Month,INTEREST RATE,coor,CURRENT PRINCIPAL AND INTEREST
,,,,mean,first,mean
0,AK,ANCHORAGE,2013-07,3.60,"(-149.835655, 61.207159)",36462.17
1,AK,ANCHORAGE,2013-09,4.08,"(-149.71576399999998, 61.225142000000005)",62664.57
2,AK,ANCHORAGE,2015-08,3.98,"(-149.738664, 61.20176)",18568.27
3,AK,FAIRBANKS,2012-01,3.90,"(-147.872217, 64.84279599999999)",6401.76
4,AK,FAIRBANKS,2013-08,3.15,"(-147.803767, 64.833887)",13938.60
...,...,...,...,...,...,...
11471,WY,ROCK SPRINGS,2014-04,4.36,"(-109.254046, 41.595486)",57410.17
11472,WY,ROCK SPRINGS,2015-07,4.26,"(-109.25347099999999, 41.595764)",90391.05
11473,WY,ROCK SPRINGS,2019-09,3.07,"(-109.2048, 41.5869)",24143.23


In [16]:
hud

,PROPERTY STATE,PROPERTY CITY,Month,INTEREST RATE,coor,CURRENT PRINCIPAL AND INTEREST
,,,,mean,first,mean
0,AK,ANCHORAGE,2013-07,3.60,"(-149.835655, 61.207159)",36462.17
1,AK,ANCHORAGE,2013-09,4.08,"(-149.71576399999998, 61.225142000000005)",62664.57
2,AK,ANCHORAGE,2015-08,3.98,"(-149.738664, 61.20176)",18568.27
3,AK,FAIRBANKS,2012-01,3.90,"(-147.872217, 64.84279599999999)",6401.76
4,AK,FAIRBANKS,2013-08,3.15,"(-147.803767, 64.833887)",13938.60
...,...,...,...,...,...,...
11471,WY,ROCK SPRINGS,2014-04,4.36,"(-109.254046, 41.595486)",57410.17
11472,WY,ROCK SPRINGS,2015-07,4.26,"(-109.25347099999999, 41.595764)",90391.05
11473,WY,ROCK SPRINGS,2019-09,3.07,"(-109.2048, 41.5869)",24143.23


In [17]:
def select_lag(data1,data2,column):
    data=data2.merge(data1,on="Month",how="left")
    lags=[-4,-3,-2,-1,0,1,2,3,4]
    corrs=[]
    corrs_abs=[]
    for i in range(len(lags)):
        name=column+"("+str(lags[i])+")"
        data[name]=data[column].shift(-lags[i])
        corrs.append(data[name].corr(data['INTEREST RATE','mean'],method ='pearson'))
        corrs_abs.append(abs(corrs[i]))
    max_i=corrs_abs.index(max(corrs_abs))
    if lags[max_i]==0:
        new_name=column
    else:
        new_name=column+"("+str(lags[max_i])+")"
    data1=data1.merge(data[[new_name,"Month"]],on="Month")
    col_name="Corr with "+new_name
    data1[col_name]=corrs[max_i]
    return data1

In [18]:
def add_rate(data1,data2,column):
    data=data2.merge(data1,on="Month",how="left")
    corr=data[column].corr(data['INTEREST RATE','mean'],method ='pearson')
    data1=data1.merge(data[[column,"Month"]],on="Month")
    col_name="Corr with "+column
    data1[col_name]=corr
    return data1

In [19]:
at_vol=pd.read_csv("../auto/vol_data_AUT.csv")
at_vol["date"]=at_vol["date"].map(lambda x:pd.to_datetime(x,format='%Y-%m'))
at_vol["Month"]=at_vol["date"].map(lambda x:x.to_period("M"))
at_vol=at_vol.drop(columns={"month",'date'})
at_vol=at_vol.rename(columns={"vol":"auto_vol"})


In [20]:
cred=pd.read_csv("../credit card/tightness.csv")
cred["date"]=cred["date"].map(lambda x:pd.to_datetime(x,format="%Y-%m"))
cred["Month"]=cred["date"].map(lambda x: x.to_period("M"))
cred=cred.drop(columns={"date","month"})
cred=cred.rename(columns={"tightness_index":"credit_tightness"})


In [21]:
stu=pd.read_csv("../student/vol_data_STU.csv")
stu["date"]=stu["date"].map(lambda x:pd.to_datetime(x,format="%Y-%m"))
stu["Month"]=stu["date"].map(lambda x:x.to_period("M"))
stu=stu.drop(columns={"month",'date'})
stu=stu.rename(columns={"vol":"student_vol"})


In [22]:
mo1=pd.read_csv("../mortgage/metro30_89.csv")
mo2=pd.read_csv("../mortgage/metro90.csv")

In [23]:
mo1["Name"]=mo1["Name"].map(lambda x:str.upper(x))
mo2["Name"]=mo2["Name"].map(lambda x:str.upper(x))
mo1=mo1[mo1["RegionType"]=="MetroArea"]
mo2=mo2[mo2["RegionType"]=="MetroArea"]
mo1=mo1.drop(columns=["CBSACode","RegionType"])
mo2=mo2.drop(columns=["CBSACode","RegionType"])
mo1["PROPERTY STATE"]=mo1["Name"].map(lambda x:str.split(x,",")[-1])
mo2["PROPERTY STATE"]=mo2["Name"].map(lambda x:str.split(x,",")[-1])
mo1["PROPERTY CITY"]=mo1["Name"].map(lambda x:str.split(x,",")[0])
mo2["PROPERTY CITY"]=mo2["Name"].map(lambda x:str.split(x,",")[0])


In [24]:
states=hud_cat["PROPERTY STATE"].unique()
cities=hud_cat["PROPERTY CITY"].unique()

In [25]:
month=pd.date_range(start="2008-01", end="2019-04",freq="M").to_period("M")

In [26]:
# li_st=[]
# cols = [i for i in mo1.columns if i not in ['Name', 'PROPERTY CITY','PROPERTY STATE']]
# df1=pd.DataFrame()
# for j in range(len(cities)):
#     sub=mo1[(mo1["PROPERTY CITY"].map(lambda x: cities[j] in(x)))]
#     for i in range(len(states)):
#         if len(sub[sub["PROPERTY STATE"].map(lambda x: states[i] in(x))])==1:
#             a=sub[sub["PROPERTY STATE"].map(lambda x: states[i] in(x))]
#             df=pd.DataFrame(month).rename(columns={0:"Month"})
#             b=np.transpose(a[cols]).reset_index()
#             b.columns=("Month","mortgage 30-89")
#             b["Month"]=b["Month"].map(lambda x:pd.to_datetime(x,format='%Y-%m'))
#             b["Month"]=b["Month"].map(lambda x:x.to_period("M"))
#             df=df.merge(b,on="Month")
#             df["PROPERTY STATE"]=states[i]
#             df["PROPERTY CITY"]=cities[j]
#             li_st.append(df)
# df1=pd.concat(li_st).reset_index(drop=True)
    


In [27]:
# li_st2=[]
# cols = [i for i in mo2.columns if i not in ['Name', 'PROPERTY CITY','PROPERTY STATE']]
# df2=pd.DataFrame()
# for j in range(len(cities)):
#     sub=mo2[(mo2["PROPERTY CITY"].map(lambda x: cities[j] in(x)))]
#     for i in range(len(states)):
#         if len(sub[sub["PROPERTY STATE"].map(lambda x: states[i] in(x))])==1:
#             a=sub[sub["PROPERTY STATE"].map(lambda x: states[i] in(x))]
#             df=pd.DataFrame(month).rename(columns={0:"Month"})
#             b=np.transpose(a[cols]).reset_index()
#             b.columns=("Month","mortgage 90")
#             b["Month"]=b["Month"].map(lambda x:pd.to_datetime(x,format='%Y-%m'))
#             b["Month"]=b["Month"].map(lambda x:x.to_period("M"))
#             df=df.merge(b,on="Month")
#             df["PROPERTY STATE"]=states[i]
#             df["PROPERTY CITY"]=cities[j]
#             li_st2.append(df)
# df2=pd.concat(li_st2).reset_index(drop=True)

In [28]:

# mortgage=df1.merge(df2,on=["PROPERTY STATE","PROPERTY CITY","Month"])
# mortgage=mortgage[["PROPERTY STATE","PROPERTY CITY","Month","mortgage 30-89","mortgage 90"]]

# mortgage.to_csv("mortgage_all.csv",index=False)

In [29]:
mortgage=pd.read_csv("mortgage_all.csv")
mortgage["Month"]=mortgage["Month"].map(lambda x:pd.to_datetime(x,format="%Y-%m")).map(lambda x:x.to_period("M"))

In [30]:

# df_list=[]
# df_s=pd.DataFrame()
# for i in range(len(cities)):
#     df=hud[hud["PROPERTY CITY"]==cities[i]]
#     df["Month"]=df["Month"].map(lambda x:pd.to_datetime(x,format='%Y-%m'))
#     df["Month"]=df["Month"].map(lambda x: x.to_period("M"))

#     df=df.merge(libor["ON"],on="Month")
#     df["spread between libor ON"]=df["INTEREST RATE"]-df["ON"]
#     df["Corr with libor ON"]=df["ON"].corr(df["INTEREST RATE"])
    
#     df=df.merge(libor["1W"],on="Month")
#     df["spread between libor 1W"]=df["INTEREST RATE"]-df["1W"]
#     df["Corr with libor 1W"]=df["1W"].corr(df["INTEREST RATE"])
    
#     df=df.merge(libor["1M"],on="Month")
#     df["spread between libor 1M"]=df["INTEREST RATE"]-df["1M"]
#     df["Corr with libor 1M"]=df["1M"].corr(df["INTEREST RATE"])
    
#     df=df.merge(libor["2M"],on="Month")
#     df["spread between libor 2M"]=df["INTEREST RATE"]-df["2M"]
#     df["Corr with libor 2M"]=df["2M"].corr(df["INTEREST RATE"])
    
#     df=df.merge(libor["3M"],on="Month")
#     df["spread between libor 3M"]=df["INTEREST RATE"]-df["3M"]
#     df["Corr with libor 3M"]=df["3M"].corr(df["INTEREST RATE"])
    
#     df=df.merge(libor["6M"],on="Month")
#     df["spread between libor 6M"]=df["INTEREST RATE"]-df["6M"]
#     df["Corr with libor 6M"]=df["6M"].corr(df["INTEREST RATE"])
    
#     df=df.merge(libor["12M"],on="Month")
#     df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
#     df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])

#     df=df.merge(frame_trea["1 Mo"],on="Month")
#     df["spread between 1 Mo treasury"]=df["INTEREST RATE"]-df["1 Mo"]
#     df["Corr with 1 Mo treasury"]=df["1 Mo"].corr(df["INTEREST RATE"])
    
#     df=df.merge(frame_trea["6 Mo"],on="Month")
#     df["spread between 6 Mo treasury"]=df["INTEREST RATE"]-df["6 Mo"]
#     df["Corr with 6 Mo treasury"]=df["6 Mo"].corr(df["INTEREST RATE"])
    
#     df=df.merge(frame_trea["1 Yr"],on="Month")
#     df["spread between 1 Yr treasury"]=df["INTEREST RATE"]-df["1 Yr"]
#     df["Corr with 1 Yr treasury"]=df["1 Yr"].corr(df["INTEREST RATE"])
    
#     df=df.merge(frame_trea["5 Yr"],on="Month")
#     df["spread between 5 Yr treasury"]=df["INTEREST RATE"]-df["5 Yr"]
#     df["Corr with 5 Yr treasury"]=df["5 Yr"].corr(df["INTEREST RATE"])
    
#     df=df.merge(frame_trea["10 Yr"],on="Month")
#     df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
#     df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])
    
#     df=df.merge(frame_trea["20 Yr"],on="Month")
#     df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
#     df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])
    
#     df=df.merge(frame_trea["30 Yr"],on="Month")
#     df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
#     df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
    
#     df=select_lag(df,at_vol,"auto_vol")
    
#     df=select_lag(df,stu,"student_vol")
    
#     df=select_lag(df,cred,"credit_tightness")
    
#     df=select_lag(df,mortgage[mortgage["PROPERTY CITY"]==cities[i]],"mortgage 30-89")
    
#     df=select_lag(df,mortgage[mortgage["PROPERTY CITY"]==cities[i]],"mortgage 90")

#     df_list.append(df)
#     del df
#     gc.collect()
    
    
# df_s=pd.concat(df_list).reset_index(drop=True)


# df_s.to_csv("City Data of HUD_month_all.csv",index=False)

In [31]:
def concat_ordered_columns(frames): 
    columns_ordered = [] 
    for frame in frames: 
        columns_ordered.extend(x for x in frame.columns if x not in columns_ordered) 
    final_df = pd.concat(frames)  
    return final_df[columns_ordered]  

In [32]:
hud_cat["Category"]=hud_cat["Category"].map(lambda x:x.replace("Refinancing/Purchase of Apartments  & Coops","Refinancing_Purchase of Apartments_Coops")).map(lambda x:x.replace(
"Improvements/Additions for Apartments / Coops ","Improvements_Additions for Apartments_Coops"))

In [33]:
categories=["Improvements_Additions for Apartments_Coops","Risk Sharing","New Construction","Refinancing_Purchase of Apartments_Coops",
     "Refinancing of Apt","Other FHA","Health Care Facilities",
     "Hospitals and Group Practices"]

for i in range(len(categories)):
    hud_sub=hud_cat[hud_cat["Category"].str.contains(categories[i],regex=False)]
#     hud_sub["Month"]=hud_sub["Month"].map(lambda x: pd.to_datetime(x,format='%Y-%m'))
#     hud_sub["Month"]=hud_sub["Month"].map(lambda x:x.to_period("M"))
    hud_sub=hud_sub.groupby(["PROPERTY STATE","PROPERTY CITY","Month"]).agg({'INTEREST RATE':['mean'],'coor':['first'],'CURRENT PRINCIPAL AND INTEREST':['mean']})
    hud_sub=hud_sub.reset_index(["PROPERTY STATE","PROPERTY CITY","Month"])
    hud_sub=hud_sub[hud_sub["PROPERTY CITY"].isin(cities)]
    hud_sub["Category"]=categories[i]
    df_list=[]
    df_s=pd.DataFrame()

    for j in range(len(cities)):
        sub_sub=hud_sub[hud_sub["PROPERTY CITY"]==cities[j]]
        state=sub_sub["PROPERTY STATE"].unique()
        a=mortgage[mortgage["PROPERTY CITY"]==cities[j]]
        for k in range(len(state)):
            df=sub_sub[sub_sub["PROPERTY STATE"]==state[k]]
            if (len(df)>=5):
                
                df=add_rate(df,libor,"ON")
                

                df=add_rate(df,libor,"1W")

                df=add_rate(df,libor,"1M")
                
                df=add_rate(df,libor,"2M")
                
                df=add_rate(df,libor,"3M")
                
                df=add_rate(df,libor,"6M")
                
                df=add_rate(df,libor,"12M")

                df=add_rate(df,frame_trea,"1 Mo")
                
                df=add_rate(df,frame_trea,"6 Mo")
                
                df=add_rate(df,frame_trea,"1 Yr")
                
                df=add_rate(df,frame_trea,"5 Yr")
                
                df=add_rate(df,frame_trea,"10 Yr")
                
                df=add_rate(df,frame_trea,"20 Yr")
                
                df=add_rate(df,frame_trea,"30 Yr")

                df=select_lag(df,at_vol,"auto_vol")
            
                df=select_lag(df,stu,"student_vol")

                df=select_lag(df,cred,"credit_tightness")
                
                b=a[a["PROPERTY STATE"]==state[k]]
                df=select_lag(df,b.reset_index(drop=True),"mortgage 30-89")
            
                df=select_lag(df,b.reset_index(drop=True),"mortgage 90")
    
                name="All/Category/"+categories[i]+"/"+cities[j]+"_"+state[k]+".csv"
                name_json="All/Category/"+categories[i]+"/"+cities[j]+"_"+state[k]+".json"

            
                df["long"]=df["coor","first"].map(lambda x:x[0])
                df["lat"]=df["coor","first"].map(lambda x:x[1])
                df.to_csv(name)
                
                data=df.to_json()
                
                file = open(name_json,'w',encoding='utf-8')
                json.dump(data,file,ensure_ascii=False)


C:\Users\yz402\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\yz402\Anaconda3\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)
C:\Users\yz402\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\yz402\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\yz402\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [34]:
hud_cat["Category"].unique()

array(["['Refinancing_Purchase of Apartments_Coops', 'Refinancing of Apt', 'Improvements_Additions for Apartments_Coops', 'Other FHA']",
       "['New Construction', 'Refinancing_Purchase of Apartments_Coops', 'Refinancing of Apt', 'Improvements_Additions for Apartments_Coops']",
       "['Refinancing_Purchase of Apartments_Coops', 'Refinancing of Apt', 'Improvements_Additions for Apartments_Coops', 'Health Care Facilities']",
       "['Risk Sharing', 'Refinancing_Purchase of Apartments_Coops', 'Refinancing of Apt', 'Improvements_Additions for Apartments_Coops']",
       "['Refinancing_Purchase of Apartments_Coops', 'Refinancing of Apt', 'Improvements_Additions for Apartments_Coops', 'Hospitals and Group Practices']",
       "['Refinancing_Purchase of Apartments_Coops', 'Refinancing of Apt', 'Improvements_Additions for Apartments_Coops']",
       "['Refinancing of Apt', 'Improvements_Additions for Apartments_Coops']",
       "['Refinancing_Purchase of Apartments_Coops', 'Refinancing of

In [35]:
hud_all=hud_cat
hud_all=hud_all.groupby(["PROPERTY STATE","PROPERTY CITY","Month"]).agg({'INTEREST RATE':['mean'],'coor':['first'],'CURRENT PRINCIPAL AND INTEREST':['mean']})
hud_all=hud_all.reset_index(["PROPERTY STATE","PROPERTY CITY","Month"])
hud_all=hud_all[hud_all["PROPERTY CITY"].isin(cities)]
hud_all["Category"]="All"

# hud_all["Month"]=hud_all["Month"].map(lambda x: pd.to_datetime(x,format='%Y-%m'))
# hud_all["Month"]=hud_all["Month"].map(lambda x:x.to_period("M"))

In [36]:
hud_all["Month"]

0        2013-07
1        2013-09
2        2015-08
3        2012-01
4        2013-08
          ...   
11471    2014-04
11472    2015-07
11473    2019-09
11474    2018-09
11475    2019-06
Name: Month, Length: 11476, dtype: period[M]

In [37]:

df_list=[]
df_s=pd.DataFrame()

for j in range(len(cities)):
    sub=hud_all[hud_all["PROPERTY CITY"]==cities[j]]
    state=sub["PROPERTY STATE"].unique()
    a=mortgage[mortgage["PROPERTY CITY"]==cities[j]]
    for k in range(len(state)):
        df=sub[sub["PROPERTY STATE"]==state[k]]
        if (len(df)>=5):
            
            df=add_rate(df,libor,"ON")
                

            df=add_rate(df,libor,"1W")

            df=add_rate(df,libor,"1M")
            
            df=add_rate(df,libor,"2M")

            df=add_rate(df,libor,"3M")

            df=add_rate(df,libor,"6M")

            df=add_rate(df,libor,"12M")

            df=add_rate(df,frame_trea,"1 Mo")

            df=add_rate(df,frame_trea,"6 Mo")

            df=add_rate(df,frame_trea,"1 Yr")

            df=add_rate(df,frame_trea,"5 Yr")

            df=add_rate(df,frame_trea,"10 Yr")

            df=add_rate(df,frame_trea,"20 Yr")

            df=add_rate(df,frame_trea,"30 Yr")

            df=select_lag(df,at_vol,"auto_vol")

            df=select_lag(df,stu,"student_vol")

            df=select_lag(df,cred,"credit_tightness")

            b=a[a["PROPERTY STATE"]==state[k]]

            df=select_lag(df,b.reset_index(drop=True),"mortgage 30-89")

            df=select_lag(df,b.reset_index(drop=True),"mortgage 90")

            name="All/Category/"+"All"+"/"+cities[j]+"_"+state[k]+".csv"
            
            df["long"]=df["coor","first"].map(lambda x:x[0])
            df["lat"]=df["coor","first"].map(lambda x:x[1])
            
            df.to_csv(name)
            name_json="All/Category/"+"All"+"/"+cities[j]+"_"+state[k]+".json"

            data=df.to_json()

            file = open(name_json,'w',encoding='utf-8')
            json.dump(data,file,ensure_ascii=False)


In [38]:
# categories=["Risk Sharing","New Construction","Refinancing_Purchase of Apartments_Coops",
#      "Refinancing of Apt","Improvements_Additions for Apartments_Coops","Other FHA","Health Care Facilities",
#      "Hospitals and Group Practices"]

# hud_sub=hud_cat[hud_cat["Category"].str.contains("Health Care Facilities",regex=False)]
# hud_sub["Month"]=hud_sub["Month"].map(lambda x: pd.to_datetime(x,format='%Y-%m'))
# hud_sub["Month"]=hud_sub["Month"].map(lambda x:x.to_period("M"))
# hud_sub=hud_sub.groupby(["PROPERTY STATE","PROPERTY CITY","Month"])[["INTEREST RATE","CURRENT PRINCIPAL AND INTEREST"]].mean()
# hud_sub=hud_sub.reset_index(["PROPERTY STATE","PROPERTY CITY","Month"])

# test=hud_sub[(hud_sub["PROPERTY CITY"]=="CHICAGO") & (hud_sub["PROPERTY STATE"]=="IL")]